## 文本多路召回与重排序
### 任务
- 任务说明：实现多种文本编码和检索逻辑，并进行重排序
- 任务要求：
    - 结合文本索引和向量检索结果
    - 加载重排序模型，对检索进行重排序
- 打卡要求：完成多路召回与重排序，与任务5精度进行对比

### 代码

In [3]:
import jieba, json, pdfplumber
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
from rank_bm25 import BM25Okapi


In [4]:

questions = json.load(open("./data/questions.json"))

pdf = pdfplumber.open("./data/初赛训练数据集.pdf")
pdf_content = []
for page_idx in range(len(pdf.pages)):
    pdf_content.append({
        'page': 'page_' + str(page_idx + 1),
        'content': pdf.pages[page_idx].extract_text()
    })


In [7]:
# 加载重排序模型

import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('/root/code/Xorbits/bge-small-zh-v1___5/')
rerank_model = AutoModelForSequenceClassification.from_pretrained('/root/code/Xorbits/bge-small-zh-v1___5/')
rerank_model.cuda()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /root/code/Xorbits/bge-small-zh-v1___5/ and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 512, padding_idx=0)
      (position_embeddings): Embedding(512, 512)
      (token_type_embeddings): Embedding(2, 512)
      (LayerNorm): LayerNorm((512,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-3): 4 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=512, out_features=512, bias=True)
              (key): Linear(in_features=512, out_features=512, bias=True)
              (value): Linear(in_features=512, out_features=512, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=512, out_features=512, bias=True)
              (LayerNorm): LayerNorm((512,), eps=1e-12, e

In [8]:
pdf_content_words = [jieba.lcut(x['content']) for x in pdf_content]
bm25 = BM25Okapi(pdf_content_words)

In [19]:
pairs

[['请问在汽车技术中，1牛米（Nm）的力矩等于多少牛顿（N）乘以米（m）？',
  '泊车\n泊车辅助系统检测范围\n检测区域 检测范围（米）\n后部两侧 0.3\n后部中央 1.5\n前部两侧 0.3\n前部中央 0.9\n泊车辅助系统局限性\n关于泊车辅助系统局限性，请参见泊车辅助传感器章节。 车辆启动后，您将换挡杆切换至倒挡（R）进行倒车时，后泊车辅助\n系统自动启用，声音警告信号在车辆离后方物体1.5m时启用。\n泊车辅助系统\n警告！\n车辆低速行驶时，泊车辅助系统通过雷达传感器检测车辆后方是否有\n障碍物，并通过声音提示驾驶员车辆与障碍物之间的距离。 ■ 作为驾驶员，您应遵守相关法律要求，并对安全停车承担全部责\n任。\n■ 泊车辅助系统是一种辅助功能，在使用过程中您仍需注意观察周\n围环境。\n225'],
 ['请问在汽车技术中，1牛米（Nm）的力矩等于多少牛顿（N）乘以米（m）？',
  '技术资料\n说明！ 汽车轴荷（满载状态）\n□ 请保持挡风玻璃洁净、干燥；请不要在安装汽车电子标识处贴膜 前轴最大轴荷 后轴最大轴荷\n或粘贴金属材料，确保汽车电子标识的规范安装和数据的有效读 车型 （kg） （kg）\n取。\nMR6453DCHEV02 1214 1116\n车辆参数\n发动机规格\n车辆尺寸参数（整备质量） 最大净功率 额定功率 最大扭矩\n发动机\n（kW/rpm） （kW/rpm） （Nm/rpm）\n尺寸 单位（mm）\n225/（2500-\nDHE15-ESZ 108/5500 110/5500\n长度 4549 4000）\n高度 1689（a）\n电机规格\n宽度 1860（b）\n（a）：测量高度包含多波段天线。 电机类型 峰值功率（kW）\n（b）：车辆宽度为不包括外后视镜的车宽。\n永磁同步电机 100\n汽车重量\n动力电池规格\n最大允许总重量\n车型 整备质量（kg）\n（kg）\n电池类型 容量（Ah） 额定电压（V）\nMR6453DCHEV02 1890 2330\n锂离子电池 51 347.5\n349'],
 ['请问在汽车技术中，1牛米（Nm）的力矩等于多少牛顿（N）乘以米（m）？',
  '技术资料\n公制术语\n术语 说明\n术语 说明\nN 牛\n% 百分比 Nm 牛米\nX:1 比值 V 伏特

In [5]:
for query_idx in range(len(questions)):
    # 首先进行BM25检索
    doc_scores = bm25.get_scores(jieba.lcut(questions[query_idx]["question"]))
    max_score_page_idxs = doc_scores.argsort()[-3:]
    	
    # top3进行重排序
    pairs = []
    for idx in max_score_page_idxs:
        pairs.append([questions[query_idx]["question"], pdf_content[idx]['content']])

    inputs = tokenizer(pairs, padding=True, truncation=True, return_tensors='pt', max_length=512)
    with torch.no_grad():
        inputs = {key: inputs[key].cuda() for key in inputs.keys()}
        scores = rerank_model(**inputs, return_dict=True).logits.view(-1, ).float()

    max_score_page_idx = max_score_page_idxs[scores.cpu().numpy().argmax()]
    questions[query_idx]['reference'] = 'page_' + str(max_score_page_idx + 1)

with open('submit.json', 'w', encoding='utf8') as up:
    json.dump(questions, up, ensure_ascii=False, indent=4)

/root/.conda/envs/llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/root/.conda/envs/llm/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/root/.conda/envs/llm/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/root/.conda/envs/llm/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree